In [75]:
import pandas as pd
txf_li=pd.read_csv('txf_li.csv')
txf_li_revised=txf_li[['TRR_ID','TRR_FOL_ID','TFL_FOL_CD','PX_ID','TX_ID','TFL_CAD','TFL_CREAT','TFL_INR','TFL_TOT_BILI','TFL_SGPT','TFL_SGOT','TFL_REJ_EVENT_NUM','TFL_DIAB_DURING_FOL','TFL_INSULIN_DEPND','TFL_FAIL_BILIARY','TFL_GRAFT_STAT','TFL_HOSP','TFL_IMMUNO_DISCONT','TFL_MALIG','TFL_MALIG_LYMPH','TFL_MALIG_RECUR_TUMOR','TFL_MALIG_TUMOR','TFL_PX_NONCOMP','TFL_REJ_TREAT','REC_TX_DT','TFL_PX_STAT','TFL_PX_STAT_DT','TFL_COD','TFL_FAIL_DT','TFL_FAIL_HEP_DENOVO','TFL_FAIL_HEP_RECUR','TFL_FAIL_INFECT','TFL_FAIL_PRIME_GRAFT_FAIL','TFL_FAIL_PX_NONCOMP','TFL_FAIL_RECUR_DISEASE','TFL_FAIL_REJ_ACUTE','TFL_FAIL_REJ_CHRONIC','TFL_FAIL_VASC_THROMB','TFL_ACUTE_REJ_EPISODE','TFL_ACUTE_REJ_BIOPSY_CONFIRMED','TFL_ALBUMIN','TFL_ALKPHOS','TFL_ANTIVRL_THERAPY','TFL_ANTIVRL_THERAPY_TY','TFL_BMI','TFL_WGT_KG','TFL_HGT_CM','TFL_CMV_IGG','TFL_CMV_IGM','TFL_IMMUNO_MAINT_MEDS','REC_TX_ORG_TY','TFL_COD2','TFL_COD3','TFL_ANTIVRL_THERAPY_TY_ACYCLOVIR','TFL_ANTIVRL_THERAPY_TY_CYTOGAM','TFL_ANTIVRL_THERAPY_TY_GAMIMUNE','TFL_ANTIVRL_THERAPY_TY_GAMMAGARD','TFL_ANTIVRL_THERAPY_TY_CYTOVENE','TFL_ANTIVRL_THERAPY_TY_VALCYTE','TFL_ANTIVRL_THERAPY_TY_HBIG','TFL_ANTIVRL_THERAPY_TY_FLU_VAC','TFL_ANTIVRL_THERAPY_TY_EPIVIR','TFL_ANTIVRL_THERAPY_TY_OTHER','TFL_ANTIVRL_THERAPY_TY_VALTREX','TFL_HOSP_NUM']]



In [76]:
# To get number of patients whom entry before 2003, after 2013, and multiple transplants, and retransplanted patient:
plessthan2003=txf_li_revised.loc[(txf_li_revised['REC_TX_DT']<'2003-01-01')]
print('Number of patients with entry before Jan 2003:', len(plessthan2003['TRR_ID'].unique().tolist()))

pafter2014=txf_li_revised.loc[(txf_li_revised['REC_TX_DT']>'2013-12-31')]
print('Number of patients with entry after Dec 2013:', len(pafter2014['TRR_ID'].unique().tolist()))

otherthanliver=txf_li_revised.loc[(txf_li_revised['REC_TX_ORG_TY']!='LI')]
print('Number of patients with transplant other than liver/multiple transplants:', len(otherthanliver['TRR_ID'].unique().tolist()))

retransplant=txf_li_revised.loc[(txf_li_revised['TFL_PX_STAT']=='R')]
print('Number of retransplant patient:', len(retransplant['TRR_ID'].unique().tolist()))
#R is number of retransplant patient
R=retransplant['TRR_ID'].unique().tolist()

#to exclude data for patients with entry less than 2003:
txf_li_revised=txf_li_revised.loc[~(txf_li_revised['REC_TX_DT']<'2003-01-01')]

#to exlude data for patients with entry after 2014
txf_li_revised=txf_li_revised.loc[~(txf_li_revised['REC_TX_DT']>'2013-12-31')]

# to keep only liver transplant:
txf_li_revised=txf_li_revised.loc[(txf_li_revised['REC_TX_ORG_TY']=='LI')]

#To exclude retransplant patient:
for e in R:
    txf_li_revised.drop(txf_li_revised.loc[txf_li_revised['TRR_ID']==e].index, inplace=True)



#Filtering out graft failure, infections,malignancy,cardiac,other_death 
#here below are the tfl_cod code for graft failure,infections,malignancy,cardiac,other death

graft_failure=[4955.0,4956.0,4957.0,4958.0,4600.0,4601.0,4602.0,4603.0,4604.0,4605.0,4606.0,4610.0]
infections=[4800.0,4801.0,4802.0,4803.0,4804.0,4805.0,4806.0,4810.0,4811.0,4660.0]
malignancy=[4850.0,4851.0,4855.0,4856.0,4860.0]
cardiac=[4246.0,4247.0,4620.0,4621.0,4622.0,4623.0,4624.0,4625.0,4626.0]
other_death=[4951.0,4952.0,4650.0,4615.0,4630.0,4631.0,4635.0,4940.0]
 
    
#To keep graft_failure patients and filter out multiple causes of deaths, and filter out patients whom died without follow up:
#To filter out mutliple causes of deaths:
graft_failure_frames = []
i = 0
for e in graft_failure:
    graft_failure_frames += [(txf_li_revised.loc[(txf_li_revised['TFL_COD'] == e) & (txf_li_revised['TFL_COD2'].isnull()) &(txf_li_revised['TFL_COD3'].isnull())])] 

all_gf=pd.concat(graft_failure_frames)

# to filter out gf_patients whom died before first follow-up:
#gf_frames is list of graft failure patients whom died after first follow up
gf_frames=[]
for e in all_gf['TRR_ID'].unique().tolist():
    gf_frames+= [(txf_li_revised.loc[(txf_li_revised['TRR_ID'] == e) & (txf_li_revised['TFL_FOL_CD'] > 3.0) &(txf_li_revised['TFL_FOL_CD'] < 10.0 )])] 

graft_failure_patients = pd.concat(gf_frames)
print('Number of graft failure patients:', len(graft_failure_patients['TRR_ID'].unique().tolist()))
 
    
    
    
#To keep infection patients and filter out multiple causes of deaths + filter out infection patients whom died before first follow up:
#to filter out multiple causes of death:
infection_frames = []
i = 0
for e in infections:
    infection_frames += [(txf_li_revised.loc[(txf_li_revised['TFL_COD'] == e) & (txf_li_revised['TFL_COD2'].isnull()) &(txf_li_revised['TFL_COD3'].isnull())])] 

all_inf=pd.concat(infection_frames)

# to filter out infection_patients whom died before first follow-up:
#inf_frames is list of infection patients whom died after first follow up
inf_frames=[]
for e in all_inf['TRR_ID'].unique().tolist():
    inf_frames+= [(txf_li_revised.loc[(txf_li_revised['TRR_ID'] == e) & (txf_li_revised['TFL_FOL_CD'] > 3.0) &(txf_li_revised['TFL_FOL_CD'] < 10.0 )])] 

infection_patients = pd.concat(inf_frames)
print('Number of infection patients:', len(infection_patients['TRR_ID'].unique().tolist()))





#To keep malig patients and filter out multiple causes of deaths + filter out malig patients whom died before first follow up:
#To filter out multiple cause of deaths
malig_frames = []
i = 0
for e in malignancy:
    malig_frames += [(txf_li_revised.loc[(txf_li_revised['TFL_COD'] == e)& (txf_li_revised['TFL_COD2'].isnull()) &(txf_li_revised['TFL_COD3'].isnull())])] 

all_malig=pd.concat(malig_frames)

# to filter out malignancy_patients whom died before first follow-up:
#m_frames is list of malignancy patients whom died after first follow up:
m_frames=[]
for e in all_malig['TRR_ID'].unique().tolist():
    m_frames+= [(txf_li_revised.loc[(txf_li_revised['TRR_ID'] == e) & (txf_li_revised['TFL_FOL_CD'] > 3.0) &(txf_li_revised['TFL_FOL_CD'] < 10.0 )])] 

malig_patients = pd.concat(m_frames)
print('Number of malig patients:', len(malig_patients['TRR_ID'].unique().tolist()))



#To keep cardiac patients and filter out causes of death, filter out cardiac patients whom died before first follow up:
#To filter out mutliple causes of death, only isolate cardiac patients:
cardiac_frames = []
i = 0
for e in cardiac:
    cardiac_frames += [(txf_li_revised.loc[(txf_li_revised['TFL_COD'] == e) & (txf_li_revised['TFL_COD2'].isnull()) &(txf_li_revised['TFL_COD3'].isnull())])] 
all_cardiac=pd.concat(cardiac_frames)
 
# to filter out cardiac_patients whom died before first follow-up:
#car_frames is list of cardiac patients whom died after first follow up: 
car_frames=[]
for e in all_cardiac['TRR_ID'].unique().tolist():
    car_frames += [(txf_li_revised.loc[(txf_li_revised['TRR_ID'] == e) & (txf_li_revised['TFL_FOL_CD'] > 3.0) &(txf_li_revised['TFL_FOL_CD'] < 10.0 )])] 

cardiac_patients = pd.concat(car_frames)
print('Number of cardiac patients:', len(cardiac_patients['TRR_ID'].unique().tolist()))



#To keep patients died for other reason than graftfailure, cardiac,malig,infection/ filter out mutliple causes of death+ filter out whom died before first follow up:
otherdeath_frames = []
i = 0
for e in other_death:
    otherdeath_frames += [(txf_li_revised.loc[(txf_li_revised['TFL_COD'] == e) & (txf_li_revised['TFL_COD2'].isnull()) &(txf_li_revised['TFL_COD3'].isnull())])] 

all_other=pd.concat(otherdeath_frames)

# to filter out other_death_patients whom died before first follow-up:
# od_frame is list of 'other death' patients whom died after first follow up:
od_frames=[]
for e in all_other['TRR_ID'].unique().tolist():
    od_frames+= [(txf_li_revised.loc[(txf_li_revised['TRR_ID'] == e) & (txf_li_revised['TFL_FOL_CD'] >3.0) &(txf_li_revised['TFL_FOL_CD'] < 10.0 )])] 

other_death_patients = pd.concat(od_frames)
print('Number of patients whom died for reasons other than infection, graft failure, malignancy,cardiac :', len(other_death_patients['TRR_ID'].unique().tolist()))


#to concat all the above patients whom died for list of reasons:cardiac, malig, infection,graft failure, and other death frame :
death_database=pd.concat(gf_frames+inf_frames+m_frames+car_frames+od_frames)


#This following list includes all patients whom died due to cardiac,graft failure,malignancy,infection:  
A=death_database['TRR_ID'].unique().tolist()


#Here is list of patients who are alive, died, na in the patient status:
B=txf_li_revised['TRR_ID'].unique().tolist()

#Here is the list of all death patients-including unknown death:
all_death=txf_li_revised.loc[(txf_li_revised['TFL_PX_STAT']=='D')]
all_death_list=all_death['TRR_ID'].unique().tolist()


#to get the patients who survive:
def comprehension(B, A):
     return [x for x in B if x not in A]
    
#C is the list of all survival patient (exclude from the all_death_list)    
C=comprehension(B,all_death_list)

# to get patients whom survive and have more than 5 years follow up:
Alive_5yearsfolup=[]
i=0
for e in C:
    Alive_5yearsfolup+= [(txf_li_revised.loc[(txf_li_revised['TRR_ID'] == e)& (txf_li_revised['TFL_FOL_CD'] > 40.0) &(txf_li_revised['TFL_FOL_CD'] < 60.0) &(txf_li_revised['TFL_PX_STAT'] =='A')])]
          
more5yfolup=pd.concat(Alive_5yearsfolup)

# D is the list of survival patient with more than 5 years follow up
D=more5yfolup['TRR_ID'].unique().tolist()
print('Number of survival patients more than 5 years follow up:', len(D))

#E is the list of survival patients with less than 5 years follow up:
E=comprehension (C,D)
print('Number of survival patients with less than 5 years follow up:', len(E))


#Final database:
#Here we want to add list A-patients with malig, graft failure, cardiac, malig with list D- patients with survival more than 5 years follow up:
Final_list=A+D
final_frames = []

for e in Final_list:
    final_frames+= [(txf_li_revised.loc[(txf_li_revised['TRR_ID'] == e)])] 
            
final=pd.concat(final_frames)

#To exclude patient lost to follow up:
final.drop(final.loc[final['TFL_PX_STAT']=='L'].index, inplace=True)

#To exclude patient not seen in the appointment:
final.drop(final.loc[final['TFL_PX_STAT']=='N'].index, inplace=True)
    
#write processsed txf_li into database:
final.to_excel("Processed_txf_li.xlsx")


Number of patients with entry before Jan 2003: 47802
Number of patients with entry after Dec 2013: 33053
Number of patients with transplant other than liver/multiple transplants: 9689
Number of retransplant patient: 6765
Number of graft failure patients: 1232
Number of infection patients: 2097
Number of malig patients: 2819
Number of cardiac patients: 1082
Number of patients whom died for reasons other than infection, graft failure, malignancy,cardiac : 525
Number of survival patients more than 5 years follow up: 35692
Number of survival patients with less than 5 years follow up: 3712


In [99]:
# To find out the differences in patients between the txf_li, tx_li, and immuno database: 
txf_li=pd.read_excel('Processed_txf_li.xlsx')
immuno=pd.read_csv('immuno.csv')
tx_li=pd.read_csv('tx_li.csv')
immuno_ID=immuno['TRR_ID'].unique().tolist()
txf_li_ID=txf_li['TRR_ID'].unique().tolist()
tx_li_ID=tx_li['TRR_ID'].unique().tolist()
fol_immuno_ID=fol_immuno['TRR_ID'].unique().tolist()

    
    
#txf_tx=comprehension(txf_li_ID, tx_li_ID)
#txf_immuno=comprehension(txf_li_ID, immuno_ID)
#tx_immuno=comprehension(immuno_ID,tx_li_ID)

# to find the common thing between 3 lists:
result=(set(txf_li_ID) & set(tx_li_ID) & set(immuno_ID))
# use this result list to match all the trr_id in the immuno dataset
len(result)

#Processed immuno database:
#Immuno database
immuno=pd.read_csv('immuno.csv')
E=[1.0,2.0,3.0,4.0,5.0,6.0,8.0,9.0,-2.0,44.0,45.0,46.0,47.0,48.0,49.0,54.0,55.0,57.0,58.0,59.0]
All_immuno_drug_code=immuno['REC_DRUG_CD'].unique().tolist()

def comprehension(All_immuno_drug_code, E):
     return [x for x in All_immuno_drug_code if x not in E]

to_remove_fromimmuno=comprehension(All_immuno_drug_code, E)

print('All immuno drug code:', All_immuno_drug_code)
print('To remove these columns:',to_remove_fromimmuno)
for e in to_remove_fromimmuno:
    immuno.drop(immuno.loc[immuno['REC_DRUG_CD']==e].index, inplace=True)

#immuno follow up id:
#txf_folupID=Processed_data['TRR_ID'].unique().tolist()

patient_immuno=immuno['TRR_ID'].unique().tolist()


#To replace drug code with the drug name:
mycophenolic_acid=[9,47,55,57]

everolimus=[45]

azathioprine=[8]

sicrolimus=[6,58]

tacrolimus=[5,53,54,59]

cyclosporine=[-2,44,46,48,3,4]

steroid=[1,2,49]

for e in mycophenolic_acid:
    immuno['REC_DRUG_CD'] = immuno['REC_DRUG_CD'].replace(e ,'mycophenolic_acid')
    
for e in sicrolimus:
    immuno['REC_DRUG_CD'] = immuno['REC_DRUG_CD'].replace(e ,'sicrolimus')
    
for e in tacrolimus: 
    immuno['REC_DRUG_CD'] = immuno['REC_DRUG_CD'].replace(e ,'tacrolimus')

for e in cyclosporine:
    immuno['REC_DRUG_CD'] = immuno['REC_DRUG_CD'].replace(e ,'cyclosporine')
    
for e in steroid:
    immuno['REC_DRUG_CD'] = immuno['REC_DRUG_CD'].replace(e ,'steroid')
    
immuno['REC_DRUG_CD'] = immuno['REC_DRUG_CD'].replace(8 ,'azathioprine')
immuno['REC_DRUG_CD'] = immuno['REC_DRUG_CD'].replace(45 ,'everolimus')

print('Current immuno_code:', immuno['REC_DRUG_CD'].unique().tolist())

#To convert immuno database:
for i in range(0, len(patient_immuno)): 
    patient_immuno[i] = int(patient_immuno[i])
print(patient_immuno[3])


#To make immuno database based on TRR_ID from txf_li database:
immuno_frames = []
i = 0
for e in result:
    immuno_frames += [(immuno.loc[(immuno['TRR_ID'] == e)])] 
            
immuno_database = pd.concat(immuno_frames)

#write immuno database into excel
#immuno_database.to_excel("Processed_immuno.xlsx")
immuno_database
len(immuno_database['TRR_ID'].unique().tolist())

#To find out the differences between txf_li database and immuno database:
D=immuno_database['TRR_ID'].unique().tolist()
len(D)
E=set(result)-set(D)
print(E)

#to drop patients not found in the tx_li database:
for e in F:
    immuno_database.drop(immuno_database.loc[immuno_database['TRR_ID'] == e].index, inplace=True)

#write to excel
immuno_database.to_excel("Processed_immuno.xlsx")


All immuno drug code: [9, 49, -2, -1, 16, 4, 14, 5, 8, 3, 42, 97, 98, 6, 43, 41, 15, 21, 11, 10, 40, 45, 28, 27, 46, 12, 30, 47, 44, 19, -3, 23, 25, 48, 20, 50, 52, 7, 24, 51, 26, 53, 13, 22, 55, 54, 56, 57, 59, 58, 1, 2]
To remove these columns: [-1, 16, 14, 42, 97, 98, 43, 41, 15, 21, 11, 10, 40, 28, 27, 12, 30, 19, -3, 23, 25, 20, 50, 52, 7, 24, 51, 26, 53, 13, 22, 56]
Current immuno_code: ['mycophenolic_acid', 'steroid', 'cyclosporine', 'tacrolimus', 'azathioprine', 'sicrolimus', 'everolimus']
85


In [121]:
#Process tx_li
import pandas as pd

tx_li=pd.read_csv('tx_li.csv')
#tx_li_revised=tx_li[['TRR_ID','TX_ID','PX_ID','REC_LIFE_SUPPORT_OTHER','CAN_AGE_AT_LISTING','CAN_GENDER','DON_GENDER','CAN_LAST_SERUM_SODIUM','CAN_LAST_SERUM_CREAT','CAN_LAST_SRTR_LAB_MELD','CAN_PREV_HL','CAN_PREV_HR','CAN_PREV_IN','CAN_PREV_KI','CAN_PREV_KP','CAN_PREV_LI','CAN_PREV_LU','CAN_PREV_PA','CAN_PREV_TX','DON_AGE','REC_AGE_AT_TX','REC_POSTX_LOS','REC_PREV_HL','REC_PREV_HR','REC_PREV_IN','REC_PREV_KI','REC_PREV_KP','REC_PREV_LI','REC_PREV_LU','REC_PREV_PA','REC_FAIL_BILIARY','CAN_EDUCATION','CAN_ANGINA','CAN_CEREB_VASC','CAN_DRUG_TREAT_COPD','CAN_DRUG_TREAT_HYPERTEN','CAN_PERIPH_VASC','CAN_PULM_EMBOL','CAN_MALIG','DON_TY','REC_FAIL_HEP_DENOVO','REC_FAIL_HEP_RECUR','REC_FAIL_INFECT','REC_FAIL_PRIME_GRAFT_FAIL','REC_FAIL_RECUR_DISEASE','REC_FAIL_REJ_ACUTE','REC_FAIL_VASC_THROMB','REC_PRIMARY_PAY','REC_MALIG','REC_HIV_STAT','REC_FUNCTN_STAT','CAN_RACE_SRTR','CAN_ETHNICITY_SRTR','DON_HIST_DIAB','DON_HIST_HYPERTEN','REC_IMMUNO_MAINT_MEDS','REC_DGN','REC_TX_DT','TFL_LAFUDATE','TFL_LASTATUS','REC_ACUTE_REJ_EPISODE','REC_MED_COND','TFL_COD','TFL_DEATH_DT','CAN_DIAB','CAN_DIAB_TY','REC_FAIL_DT','REC_DISCHRG_DT','REC_DISCHRG_SGOT','REC_DISCHRG_SGPT','REC_DISCHRG_ALBUMIN','REC_DISCHRG_INR','REC_DISCHRG_ALKPHOS','REC_DISCHRG_CREAT','CAN_ANGINA_CAD','REC_ACUTE_REJ_BIOPSY_CONFIRMED','REC_ANTIVRL_THERAPY','REC_ANTIVRL_THERAPY_TY','REC_WGT_KG','REC_BMI','REC_CMV_IGG','REC_CMV_IGM','REC_HCV_STAT','REC_MALIG_TY','REC_TOLERANCE_INDUCTION_TECH','REC_VARICEAL_BLEEDING','REC_ON_VENTILATOR','REC_TIPSS','REC_INOTROP_BP_SUPPORT','REC_GRAFT_STAT','REC_LIFE_SUPPORT','CAN_TIPSS','REC_VENTILATOR']]


Unique=tx_li['REC_DGN'].unique().tolist()
non_tx_li = [e for e in Unique if e not in (4235.0,4240.0,4241.0,4242.0,4245.0,4250.0,4255.0,4260.0,4264.0,4265.0,4270.0,4451.0,4271.0,4272.0,4275.0,4280.0,4285.0,4290.0,4300.0,4301.0,4302.0,4303.0,4304.0,4305.0,4100.0,4101.0,4102.0,4104.0,4105.0,4106.0,4107.0,4108.0,4110.0,4200.0,4598.0,4510.0,4306.0,4307.0,4308.0,4315.0,4400.0,4401.0,4402.0,4403.0,4404.0,4405.0,4410.0,4201.0,4202.0,4204.0,4205.0,4206.0,4207.0,4208.0,4209.0,4210.0,4212.0,4213.0,4220.0,4420.0,4430.0,4450.0,4214.0,4215.0,4216.0,4217.0,4455.0,4500.0,4230.0,4231.0,4520.0)]
print('These are REC_DGN not included in the dataset:', non_tx_li)


A=[4235.0,4240.0,4241.0,4242.0,4245.0,4250.0,4255.0,4260.0,4264.0,4265.0,4270.0,4451.0,4271.0,4272.0,4275.0,4280.0,4285.0,4290.0,4300.0,4301.0,4302.0,4303.0,4304.0,4305.0,4100.0,4101.0,4102.0,4104.0,4105.0,4106.0,4107.0,4108.0,4110.0,4200.0,4598.0,4510.0,4306.0,4307.0,4308.0,4315.0,4400.0,4401.0,4402.0,4403.0,4404.0,4405.0,4410.0,4201.0,4202.0,4204.0,4205.0,4206.0,4207.0,4208.0,4209.0,4210.0,4212.0,4213.0,4220.0,4420.0,4430.0,4450.0,4214.0,4215.0,4216.0,4217.0,4455.0,4500.0]

for e in non_tx_li:
    tx_li.drop(tx_li.loc[tx_li['REC_DGN']==e].index, inplace=True)

#To group REC_DGN into diagnosis group:
benign_tumour=[4450.0,4451.0,4455.0]
metabolic_disease=[4300.0,4301.0,4302.0,4303.0,4304.0,4305.0,4306.0,4307.0,4308.0,4315.0]
PSC=[4240.0,4241.0,4242.0,4245.0]
PBC=[4220.0,4230.0,4231.0,4235.0]
alcoholic=[4215.0,4217.0]

print(tx_li['REC_DGN'].unique().tolist())

for e in benign_tumour:
    tx_li['REC_DGN'] = tx_li['REC_DGN'].replace(e ,'benign_tumour')
    
for e in metabolic_disease:
    tx_li['REC_DGN'] = tx_li['REC_DGN'].replace(e ,'metabolic_disease')
    
for e in PSC:
    tx_li['REC_DGN'] = tx_li['REC_DGN'].replace(e ,'PSC')
    
for e in PBC:
    tx_li['REC_DGN'] = tx_li['REC_DGN'].replace(e ,'PBC')

    
for e in alcoholic:
    tx_li['REC_DGN'] = tx_li['REC_DGN'].replace(e ,'alcoholic')  
    

# To create frames from txf_li:
txli_frames = []
i = 0
for e in result:
    txli_frames += [(tx_li.loc[(tx_li['TRR_ID'] == e)])] 
            
tx_li_database = pd.concat(txli_frames)

#to find the difference between the txf_li and tx_li:
F=set(result)-set(tx_li_database['TRR_ID'].unique().tolist())
len(F)

#to drop patients not found in immuno:
for e in E:
    tx_li_database.drop(tx_li_database.loc[tx_li_database['TRR_ID'] == e].index, inplace=True)

#to keep certain features in the database:
tx_li_revised=tx_li_database[['TRR_ID','TX_ID','PX_ID','REC_LIFE_SUPPORT_OTHER','CAN_AGE_AT_LISTING','CAN_GENDER','DON_GENDER','CAN_LAST_SERUM_SODIUM','CAN_LAST_SERUM_CREAT','CAN_LAST_SRTR_LAB_MELD','CAN_PREV_HL','CAN_PREV_HR','CAN_PREV_IN','CAN_PREV_KI','CAN_PREV_KP','CAN_PREV_LI','CAN_PREV_LU','CAN_PREV_PA','CAN_PREV_TX','DON_AGE','REC_AGE_AT_TX','REC_POSTX_LOS','REC_PREV_HL','REC_PREV_HR','REC_PREV_IN','REC_PREV_KI','REC_PREV_KP','REC_PREV_LI','REC_PREV_LU','REC_PREV_PA','REC_FAIL_BILIARY','CAN_EDUCATION','CAN_ANGINA','CAN_CEREB_VASC','CAN_DRUG_TREAT_COPD','CAN_DRUG_TREAT_HYPERTEN','CAN_PERIPH_VASC','CAN_PULM_EMBOL','CAN_MALIG','DON_TY','REC_FAIL_HEP_DENOVO','REC_FAIL_HEP_RECUR','REC_FAIL_INFECT','REC_FAIL_PRIME_GRAFT_FAIL','REC_FAIL_RECUR_DISEASE','REC_FAIL_REJ_ACUTE','REC_FAIL_VASC_THROMB','REC_PRIMARY_PAY','REC_MALIG','REC_HIV_STAT','REC_FUNCTN_STAT','CAN_RACE_SRTR','CAN_ETHNICITY_SRTR','DON_HIST_DIAB','DON_HIST_HYPERTEN','REC_IMMUNO_MAINT_MEDS','REC_DGN','REC_TX_DT','TFL_LAFUDATE','TFL_LASTATUS','REC_ACUTE_REJ_EPISODE','REC_MED_COND','TFL_COD','TFL_DEATH_DT','CAN_DIAB','CAN_DIAB_TY','REC_FAIL_DT','REC_DISCHRG_DT','REC_DISCHRG_SGOT','REC_DISCHRG_SGPT','REC_DISCHRG_ALBUMIN','REC_DISCHRG_INR','REC_DISCHRG_ALKPHOS','REC_DISCHRG_CREAT','CAN_ANGINA_CAD','REC_ACUTE_REJ_BIOPSY_CONFIRMED','REC_ANTIVRL_THERAPY','REC_ANTIVRL_THERAPY_TY','REC_WGT_KG','REC_BMI','REC_CMV_IGG','REC_CMV_IGM','REC_HCV_STAT','REC_MALIG_TY','REC_TOLERANCE_INDUCTION_TECH','REC_VARICEAL_BLEEDING','REC_ON_VENTILATOR','REC_TIPSS','REC_INOTROP_BP_SUPPORT','REC_GRAFT_STAT','REC_LIFE_SUPPORT','CAN_TIPSS','REC_VENTILATOR']]

#write it into excel
tx_li_revised.to_excel("Processed_tx_li.xlsx")


These are REC_DGN not included in the dataset: [999.0, nan, 4592.0, 4593.0, 4203.0]
[4204.0, 4401.0, 4235.0, 4270.0, 4215.0, 4214.0, 4104.0, 4216.0, 4202.0, 4300.0, 4213.0, 4260.0, 4242.0, 4451.0, 4110.0, 4450.0, 4400.0, 4210.0, 4302.0, 4212.0, 4220.0, 4241.0, 4404.0, 4255.0, 4405.0, 4245.0, 4305.0, 4315.0, 4250.0, 4100.0, nan, 4240.0, 4108.0, 4102.0, 4285.0, 4403.0, 4301.0, 4207.0, 4308.0, 4231.0, 4410.0, 4272.0, 4230.0, 4303.0, 4280.0, 4206.0, 4290.0, 4307.0, 4510.0, 4275.0, 4500.0, 4265.0, 4264.0, 4455.0, 4217.0, 4200.0, 4209.0, 4402.0, 4430.0, 4420.0, 4520.0, 4205.0, 4201.0, 4101.0, 4306.0, 4304.0, 4106.0, 4107.0, 4271.0, 4208.0, 4105.0]


In [206]:
import pandas as pd
#Fol_immuno database
fol_immuno=pd.read_csv('fol_immuno.csv')
D=[1.0,2.0,3.0,4.0,5.0,6.0,8.0,9.0,-2.0,44.0,45.0,46.0,47.0,48.0,49.0,54.0,55.0,57.0,58.0,59.0]
All_drug_code=fol_immuno['TFL_IMMUNO_DRUG_CD'].unique().tolist()

def comprehension(All_drug_code, D):
     return [x for x in All_drug_code if x not in D]

to_remove=comprehension(All_drug_code, D)

print('All immuno drug code:', All_drug_code)
print('To remove these columns:',to_remove)
for e in to_remove:
    fol_immuno.drop(fol_immuno.loc[fol_immuno['TFL_IMMUNO_DRUG_CD']==e].index, inplace=True)

#txf_folupID is the list of TRR_ID in the txf_li database:
txf_folupID=tx_li_revised['TRR_ID'].unique().tolist()

#Folimmuno_folupID is the list of TRR_ID in the fol_immuno database: 
Folimmuno_folupID=fol_immuno['TRR_FOL_ID'].unique().tolist()


#To replace drug code with the drug name:
mycophenolic_acid=[9,47,55,57]

everolimus=[45]

azathioprine=[8]

sicrolimus=[6,58]

tacrolimus=[5,53,54,59]

cyclosporine=[-2,44,46,48,3,4]

steroid=[1,2,49]

for e in mycophenolic_acid:
    fol_immuno['TFL_IMMUNO_DRUG_CD'] = fol_immuno['TFL_IMMUNO_DRUG_CD'].replace(e ,'mycophenolic_acid')
    
for e in sicrolimus:
    fol_immuno['TFL_IMMUNO_DRUG_CD'] = fol_immuno['TFL_IMMUNO_DRUG_CD'].replace(e ,'sicrolimus')
    
for e in tacrolimus: 
    fol_immuno['TFL_IMMUNO_DRUG_CD'] = fol_immuno['TFL_IMMUNO_DRUG_CD'].replace(e ,'tacrolimus')

for e in cyclosporine:
    fol_immuno['TFL_IMMUNO_DRUG_CD'] = fol_immuno['TFL_IMMUNO_DRUG_CD'].replace(e ,'cyclosporine')
    
for e in steroid:
    fol_immuno['TFL_IMMUNO_DRUG_CD'] = fol_immuno['TFL_IMMUNO_DRUG_CD'].replace(e ,'steroid')
    
fol_immuno['TFL_IMMUNO_DRUG_CD'] = fol_immuno['TFL_IMMUNO_DRUG_CD'].replace(8 ,'azathioprine')
fol_immuno['TFL_IMMUNO_DRUG_CD'] = fol_immuno['TFL_IMMUNO_DRUG_CD'].replace(45 ,'everolimus')
print('Current fol_immuno_code:', fol_immuno['TFL_IMMUNO_DRUG_CD'].unique().tolist())

#To convert immuno database:
for i in range(0, len(Folimmuno_folupID)): 
    Folimmuno_folupID[i] = int(Folimmuno_folupID[i])
print(Folimmuno_folupID[5])
    
#To create fol_immuno database based on TRR_ID from txf_li database (containting patients whom died due to cancer,graft failure,infection,cardiac and patients whom surive more than 5 years follow up:
fol_immuno_frames = []
i = 0
for e in Processed_txf_li['TRR_FOL_ID'].unique().tolist():
    fol_immuno_frames += [(fol_immuno.loc[(fol_immuno['TRR_FOL_ID'] == e)])] 
            
fol_immuno_database = pd.concat(fol_immuno_frames)

#Number of patients in txf_li or eligible patients:
len(txf_folupID)


graft_failure=[4955.0,4956.0,4957.0,4958.0,4600.0,4601.0,4602.0,4603.0,4604.0,4605.0,4606.0,4610.0]
infections=[4800.0,4801.0,4802.0,4803.0,4804.0,4805.0,4806.0,4810.0,4811.0,4660.0]
malignancy=[4850.0,4851.0,4855.0,4856.0,4860.0]
cardiac=[4246.0,4247.0,4620.0,4621.0,4622.0,4623.0,4624.0,4625.0,4626.0]
other_death=[4951.0,4952.0,4615.0,4630.0,4631.0,4635.0,4940.0]
renal_failure=[4650.0]

#Graft failure:
graft_failure_frames=[]
for e in graft_failure: 
    graft_failure_frames+= [txf_li.loc[(txf_li['TFL_COD'] == e)]]

graft_failure_database=pd.concat(graft_failure_frames)

print('Number of graft failure patients in SRTR dataset:',len(graft_failure_database['TRR_ID'].unique().tolist()))
graft_failure_patients=graft_failure_database['TRR_ID'].unique().tolist()



#infections
infection_frames=[]
for e in infections: 
    infection_frames+= [txf_li.loc[(txf_li['TFL_COD'] == e)]]

infection_database=pd.concat(infection_frames)

print('Number of infection patients in SRTR dataset:',len(infection_database['TRR_ID'].unique().tolist()))
infection_patients=infection_database['TRR_ID'].unique().tolist()

#cancer
cancer_frames=[]
for e in malignancy: 
    cancer_frames+= [txf_li.loc[(txf_li['TFL_COD'] == e)]]

cancer_database=pd.concat(cancer_frames)
print('Number of cancer patients in SRTR dataset:',len(cancer_database['TRR_ID'].unique().tolist()))
cancer_patients=cancer_database['TRR_ID'].unique().tolist()



#cardiac:
cardiac_frames=[]
for e in cardiac: 
    cardiac_frames+= [txf_li.loc[(txf_li['TFL_COD'] == e)]]

cardiac_database=pd.concat(cardiac_frames)
print('Number of cardiac patients in SRTR dataset:',len(cardiac_database['TRR_ID'].unique().tolist()))
cardiac_patients=cardiac_database['TRR_ID'].unique().tolist()




#other death:
other_death_frames=[]
for e in other_death: 
    other_death_frames+= [txf_li.loc[(txf_li['TFL_COD'] == e)]]

other_death_database=pd.concat(other_death_frames)

print('Number of other death patients in SRTR dataset:',len(other_death_database['TRR_ID'].unique().tolist()))
other_death_patients=other_death_database['TRR_ID'].unique().tolist()


#renal_failure:
renal_failure_frames=[]
for e in renal_failure: 
    renal_failure_frames+= [txf_li.loc[(txf_li['TFL_COD'] == e)]]

renal_failure_database=pd.concat(renal_failure_frames)
print('Number of renal failure patients in SRTR dataset:',len(renal_failure_database['TRR_ID'].unique().tolist()))
renal_failure_patients=renal_failure_database['TRR_ID'].unique().tolist()



#Patients whom survive with more than 5 years follow up: 

B=txf_li['TRR_ID'].unique().tolist()
survived_patients=set(B)-set(graft_failure_patients)-set(infection_patients)-set(cardiac_patients)-set(other_death_patients)-set(renal_failure_patients)-set(cancer_patients)
survived_patients=list(survived_patients)
print('Number of surviving patients with more than 5 years follow up:', len(survived_patients))

infection_patients==infection_database['TRR_ID'].unique().tolist()

cancer_patients=cancer_database['TRR_ID'].unique().tolist()

graft_failure_patients=graft_failure_database['TRR_ID'].unique().tolist()

cardiac_patients=cardiac_database['TRR_ID'].unique().tolist()

other_death_patients=other_death_database['TRR_ID'].unique().tolist()

renal_failure_patients=renal_failure_database['TRR_ID'].unique().tolist()

#test trial:
test_data=pd.read_excel('my_test_data.xlsx')

infection_patients==infection_database['TRR_ID'].unique().tolist()

cancer_patients=cancer_database['TRR_ID'].unique().tolist()

graft_failure_patients=graft_failure_database['TRR_ID'].unique().tolist()

cardiac_patients=cardiac_database['TRR_ID'].unique().tolist()

other_death_patients=other_death_database['TRR_ID'].unique().tolist()

renal_failure_patients=renal_failure_database['TRR_ID'].unique().tolist()


#Survival patients:
for e in survived_patients:
    Processed_txf_li.loc[Processed_txf_li['TRR_ID'] == e, 'LABEL'] = 'Survived'

#Infection_patients:
for e in infection_patients: 
    Processed_txf_li.loc[Processed_txf_li['TRR_ID'] == e, 'LABEL'] = 'Infection'
    
#Cancer_patients:
for e in cancer_patients: 
    Processed_txf_li.loc[Processed_txf_li['TRR_ID'] == e, 'LABEL'] = 'Malignancy'

#Graft_failure patients:
for e in graft_failure_patients: 
    Processed_txf_li.loc[Processed_txf_li['TRR_ID'] == e, 'LABEL'] = 'Graft_failure'

#Cardiac patients:
for e in cardiac_patients: 
    Processed_txf_li.loc[Processed_txf_li['TRR_ID'] == e, 'LABEL'] = 'Cardiac'
    
#others:
for e in other_death_patients: 
    Processed_txf_li.loc[Processed_txf_li['TRR_ID'] == e, 'LABEL'] = 'Other_Death'
    
#Renal_failure patients:

for e in renal_failure_patients: 
    Processed_txf_li.loc[Processed_txf_li['TRR_ID'] == e, 'LABEL'] = 'Renal_failure'

    
#Add colum to tx_li:
tx_li=pd.read_excel('Processed_tx_li.xlsx')

infection_patients==infection_database['TRR_ID'].unique().tolist()

cancer_patients=cancer_database['TRR_ID'].unique().tolist()

graft_failure_patients=graft_failure_database['TRR_ID'].unique().tolist()

cardiac_patients=cardiac_database['TRR_ID'].unique().tolist()

other_death_patients=other_death_database['TRR_ID'].unique().tolist()

renal_failure_patients=renal_failure_database['TRR_ID'].unique().tolist()


#Survival patients:
for e in survived_patients:
    tx_li.loc[tx_li['TRR_ID'] == e, 'LABEL'] = 'Survived'

#Infection_patients:
for e in infection_patients: 
    tx_li.loc[tx_li['TRR_ID'] == e, 'LABEL'] = 'Infection'
    
#Cancer_patients:
for e in cancer_patients: 
    tx_li.loc[tx_li['TRR_ID'] == e, 'LABEL'] = 'Malignancy'

#Graft_failure patients:
for e in graft_failure_patients: 
    tx_li.loc[tx_li['TRR_ID'] == e, 'LABEL'] = 'Graft_failure'

#Cardiac patients:
for e in cardiac_patients: 
    tx_li.loc[tx_li['TRR_ID'] == e, 'LABEL'] = 'Cardiac'
    
#others:
for e in other_death_patients: 
    tx_li.loc[tx_li['TRR_ID'] == e, 'LABEL'] = 'Other_Death'
    
#Renal_failure patients:

for e in renal_failure_patients: 
    tx_li.loc[tx_li['TRR_ID'] == e, 'LABEL'] = 'Renal_failure'


All immuno drug code: [4, 9, 49, 5, 41, 8, 46, 6, 44, 3, 97, 45, 7, 10, 11, 14, 16, 42, 43, 47, 50, 51, 48, 98, -2, -1, 30, 24, 12, 13, 20, 22, 23, 28, 52]
To remove these columns: [41, 97, 7, 10, 11, 14, 16, 42, 43, 50, 51, 98, -1, 30, 24, 12, 13, 20, 22, 23, 28, 52]
Current fol_immuno_code: ['cyclosporine', 'mycophenolic_acid', 'steroid', 'tacrolimus', 'azathioprine', 'sicrolimus', 'everolimus']
2046459


0

In [209]:
txf_li=pd.read_excel('Processed_txf_li.xlsx')
tx_li=pd.read_excel('Processed_tx_li.xlsx')
fol_immuno=pd.read_excel('Processed_fol_immuno.xlsx')

A=fol_immuno['TRR_ID'].unique().tolist()
txf_li_frames=[]
for e in A: 
    txf_li_frames+= [txf_li.loc[(txf_li['TRR_ID'] == e)]]
    
new_txf_li=pd.concat(txf_li_frames)



tx_li_frames=[]
for e in A: 
    tx_li_frames+= [tx_li.loc[(tx_li['TRR_ID'] == e)]]
    
new_tx_li=pd.concat(tx_li_frames)


5891

In [ ]:
##PROCESS TX_LI

#Columns needed to be filled with forward-filling method
Fill_columns=['CAN_AGE_AT_LISTING','CAN_LAST_SERUM_CREAT','CAN_LAST_SRTR_LAB_MELD','REC_POSTX_LOS','CAN_EDUCATION','CAN_ANGINA','CAN_CEREB_VASC','CAN_DRUG_TREAT_COPD','CAN_DRUG_TREAT_HYPERTEN','CAN_PERIPH_VASC','CAN_PULM_EMBOL','CAN_MALIG','REC_MALIG','REC_HIV_STAT','DON_HIST_DIAB','DON_HIST_HYPERTEN','CAN_DIAB','CAN_DIAB_TY','REC_DISCHRG_DT','REC_DISCHRG_SGPT','REC_DISCHRG_ALBUMIN','REC_DISCHRG_CREAT','REC_WGT_KG','REC_BMI','REC_CMV_IGG','REC_CMV_IGM','REC_HCV_STAT','CAN_TIPSS']
new_tx_li.loc[:,Fill_columns] = new_tx_li.loc[:,Fill_columns].ffill()


#Label encoding these columns(these Le_columns do not have unknown values):
Le_columns=['CAN_GENDER','DON_GENDER','DON_TY','TFL_LASTATUS','CAN_MALIG','CAN_RACE_SRTR','REC_IMMUNO_MAINT_MEDS','REC_TIPSS','REC_LIFE_SUPPORT','REC_GRAFT_STAT','LABEL','CAN_ETHNICITY_SRTR']

test_data['CAN_GENDER'] = le.fit_transform(test_data['CAN_GENDER'])

for e in Le_columns:
  test_data[e] = le.fit_transform(test_data[e]) 



##For these columns (these columns has N,Y,U,na values) convert 'U' and null values to NA:
N_Y_U_columns=['CAN_CEREB_VASC','CAN_DRUG_TREAT_COPD','CAN_DRUG_TREAT_HYPERTEN','CAN_PERIPH_VASC','CAN_PULM_EMBOL','REC_TOLERANCE_INDUCTION_TECH','REC_VARICEAL_BLEEDING','REC_ON_VENTILATOR','REC_INOTROP_BP_SUPPORT','CAN_TIPSS','REC_ANTIVRL_THERAPY','REC_MALIG']
for e in N_Y_U_columns:
    test_data.loc[test_data[e].isnull(), e] = 'NA'
    test_data.loc[test_data[e]=='U', e] = 'NA'
#Now convert these columns in the list of columns using label encoder:
for e in N_Y_U_columns:
  test_data[e] = le.fit_transform(test_data[e]) 

#REC_HIV_STAT: missing rate: 0.4725
#REC_CMV_IGM: missing rate: 0.35
#CAN_EDUCATION: missing rate: 0.28
#CAN_ANGINA_CAD:

##For these columns (these columns have N, U, ND, P, na values) group U,ND,and na together as NA:
N_U_ND_P_columns=['REC_HIV_STAT','REC_CMV_IGG','REC_CMV_IGM','REC_HCV_STAT']
for e in N_U_ND_P_columns:
    test_data.loc[test_data[e].isnull(), e] = 'NA'
    test_data.loc[test_data[e]=='U', e] = 'NA'
    test_data.loc[test_data[e]=='ND', e] = 'NA'
    
for e in N_U_ND_P_columns:
  test_data[e] = le.fit_transform(test_data[e])



#CAN_EDUCATION convert 998 and 996 and na values to value 7:
test_data.loc[test_data['CAN_EDUCATION'].isnull(), ['CAN_EDUCATION']] = 7
test_data.loc[test_data['CAN_EDUCATION']==996, ['CAN_EDUCATION']] = 7
test_data.loc[test_data['CAN_EDUCATION']==998, ['CAN_EDUCATION']] = 7



#CAN_ANGINA_CAD convert 998 and na values into 5.0:
test_data.loc[test_data['CAN_ANGINA_CAD'].isnull(), ['CAN_ANGINA_CAD']] = 5.0
test_data.loc[test_data['CAN_ANGINA_CAD']== 998.0, ['CAN_ANGINA_CAD']] = 5.0


#CAN_ANGINA:
test_data['CAN_ANGINA'] = le.fit_transform(test_data['CAN_ANGINA'])


#for these columns it has 998 which stands for unknown: 
columns_998=['DON_HIST_DIAB','DON_HIST_HYPERTEN','CAN_DIAB','CAN_DIAB_TY']

for e in columns_998:
    test_data[e] = le.fit_transform(test_data[e])

#Replace REC_DGN with names:
test_data['REC_DGN'] = test_data['REC_DGN'].replace(4212 ,'CIRRHOSIS_AUTOIMMUNE')
test_data['REC_DGN'] = test_data['REC_DGN'].replace(4404 ,'PLM_HEPATOBLASTOMA')
test_data['REC_DGN'] = test_data['REC_DGN'].replace(4204 ,'CIRRHOSIS_TYPEC')
test_data['REC_DGN'] = test_data['REC_DGN'].replace(4213 ,'CIRRHOSIS_CRYPTOGENIC')
test_data['REC_DGN'] = test_data['REC_DGN'].replace(4214 ,'NASH')
test_data['REC_DGN'] = test_data['REC_DGN'].replace(4216 ,'ALCOHOLIC_CIRRHOSIS_C')
test_data['REC_DGN'] = test_data['REC_DGN'].replace(4210 ,'CIRRHOSIS:OTHER')
test_data['REC_DGN'] = test_data['REC_DGN'].replace(4405 ,'PLM_HEMANGIOENDOTHELIOMA')
test_data['REC_DGN'] = test_data['REC_DGN'].replace(4202 ,'CIRRHOSIS_TYPE_B-')
test_data['REC_DGN'] = test_data['REC_DGN'].replace(4401 ,'HCC')
test_data['REC_DGN'] = test_data['REC_DGN'].replace(4108 ,'ETIOLOGY_UNKNOWN')
test_data['REC_DGN'] = test_data['REC_DGN'].replace(4500 ,'HYPERALIMENTATION_IND_LIVER_DISEASE')
test_data['REC_DGN'] = test_data['REC_DGN'].replace(4102 ,'AHN:TYPEB-HBSAG+')
test_data['REC_DGN'] = test_data['REC_DGN'].replace(4270 ,'BILIARYATRESIAEXTRAHEPATIC')
test_data['REC_DGN'] = test_data['REC_DGN'].replace(4255 ,'FAMILIAL_CHOLESTASIS')
test_data['REC_DGN'] = test_data['REC_DGN'].replace(4104 ,'AHN:TYPE_C')
test_data['REC_DGN'] = test_data['REC_DGN'].replace(4100 ,'AHN:DRUG_OTHER')
test_data['REC_DGN'] = test_data['REC_DGN'].replace(4403 ,'PLM_CHOLANGIOCARCINOMA')
test_data['REC_DGN'] = test_data['REC_DGN'].replace(4110 ,'AHNOTHERSPECIFY')
test_data['REC_DGN'] = test_data['REC_DGN'].replace(4400 ,'PLM_HEPATOMA')
test_data['REC_DGN'] = test_data['REC_DGN'].replace(4410 ,'PLM:OTHER_SPECIFY')
test_data['REC_DGN'] = test_data['REC_DGN'].replace(4260 ,'CHOLES_LIVER_DISEASE_OTHER')
test_data['REC_DGN'] = test_data['REC_DGN'].replace(4272 ,'BILIARY_HYPOPLASIA_ALAGILLE')
test_data['REC_DGN'] = test_data['REC_DGN'].replace(4280 ,'CONGENITAL_HEPATIC_FIBROSIS')
test_data['REC_DGN'] = test_data['REC_DGN'].replace(4209 ,'CHRONIC_ACTIVE_HEPATITIS')
test_data['REC_DGN'] = test_data['REC_DGN'].replace(4200 ,'CIRRHOSIS:DRUG/INDUST')
test_data['REC_DGN'] = test_data['REC_DGN'].replace(4285 ,'CYSTIC_FIBROSIS')
test_data['REC_DGN'] = test_data['REC_DGN'].replace(4101 ,'AHN:TYPE_A')
test_data['REC_DGN'] = test_data['REC_DGN'].replace(4206 ,'CIRRHOSIS:TYPEBANDC')
test_data['REC_DGN'] = test_data['REC_DGN'].replace(4275 ,'BILIARY_ATRESIA_OR_HYPOPLASIA')
test_data['REC_DGN'] = test_data['REC_DGN'].replace(4290 ,'BUDD-CHIARI_SYNDROME')
test_data['REC_DGN'] = test_data['REC_DGN'].replace(4420 ,'BILE_DUCT_CANCER')
test_data['REC_DGN'] = test_data['REC_DGN'].replace(4265 ,'NEONATAL_HEPATITIS')
test_data['REC_DGN'] = test_data['REC_DGN'].replace(4430 ,'SECONDARY_HEPATIC_MALIGNANCY')
test_data['REC_DGN'] = test_data['REC_DGN'].replace(4106 ,'AHN:TYPEBANDC')
test_data['REC_DGN'] = test_data['REC_DGN'].replace(4271 ,'BILIARY_HYPOPLASIA')
test_data['REC_DGN'] = test_data['REC_DGN'].replace(4207 ,'CIRRHOSIS:TYPE_BD')
test_data['REC_DGN'] = test_data['REC_DGN'].replace(4250 ,'BYLER_DISEASE')
test_data['REC_DGN'] = test_data['REC_DGN'].replace(4520 ,'TRAUMA_OTHER_SPECIFY')
test_data['REC_DGN'] = test_data['REC_DGN'].replace(4510 ,'GRAFTVS.HOST_DIS')
test_data['REC_DGN'] = test_data['REC_DGN'].replace(4402 ,'FIBROLAMELLAR')
test_data['REC_DGN'] = test_data['REC_DGN'].replace(4264 ,'NEONATAL_CHOLESTATIC_LIVER_DISEASE')
test_data['REC_DGN'] = test_data['REC_DGN'].replace(4201 ,'CIRRHOSIS:TYPE_A')
test_data['REC_DGN'] = test_data['REC_DGN'].replace(4208 ,'IRRHOSIS:CRYPTOGENIC-IDIOPATHIC')
test_data['REC_DGN'] = test_data['REC_DGN'].replace(4105 ,'AHN:TYPE_D')
test_data['REC_DGN'] = test_data['REC_DGN'].replace(4205 ,'CIRRHOSIS:TYPE_D')

test_data['REC_DGN'] = le.fit_transform(test_data['REC_DGN'])

#REC_FUNCTION_STAT: missing rate 0.3
##REC_FUNCTION_STAT
test_data.loc[test_data['REC_FUNCTN_STAT'].isnull(), ['REC_FUNCTN_STAT']] = 998
test_data.loc[test_data['REC_FUNCTN_STAT']==996, ['REC_FUNCTN_STAT']] = 998
test_data['REC_FUNCTN_STAT'] = le.fit_transform(test_data['REC_FUNCTN_STAT'])

#REC_MALIG_TY:


#Columns with various number and needed to be normalize
Normalized_columns=['CAN_LAST_SERUM_SODIUM','CAN_LAST_SERUM_CREAT','CAN_LAST_SRTR_LAB_MELD','DON_AGE','REC_AGE_AT_TX','REC_POSTX_LOS','REC_DISCHRG_SGOT','REC_DISCHRG_SGPT','REC_DISCHRG_ALBUMIN','REC_DISCHRG_INR','REC_DISCHRG_ALKPHOS','REC_DISCHRG_CREAT','REC_ANTIVRL_THERAPY_TY','REC_WGT_KG','REC_BMI','']
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
for e in Normalized_columns:
    df[e]=scaler.fit_transform(df[e])
    
#Columns I might needed to remove from the tx_li database: 
Removed_columns=['CAN_LAST_SERUM_SODIUM','REC_FAIL_BILIARY','REC_FAIL_HEP_DENOVO','REC_FAIL_HEP_RECUR','REC_FAIL_INFECT','REC_FAIL_PRIME_GRAFT_FAIL','REC_FAIL_RECUR_DISEASE','REC_FAIL_REJ_ACUTE','REC_FAIL_VASC_THROMB','REC_FAIL_DT','REC_DISCHRG_INR','REC_TOLERANCE_INDUCTION_TECH','REC_DISCHRG_ALKPHOS']

 